# Music genre classification

![](https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2020/05/music-genre-classifier-model.jpg)

## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyas.info/downloads/datasets.html) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

In [1]:
!wget http://opihi.cs.uvic.ca/sound/genres.tar.gz

--2021-02-22 20:44:15--  http://opihi.cs.uvic.ca/sound/genres.tar.gz
Resolving opihi.cs.uvic.ca (opihi.cs.uvic.ca)... 142.104.68.135
Connecting to opihi.cs.uvic.ca (opihi.cs.uvic.ca)|142.104.68.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1225571541 (1.1G) [application/x-gzip]
Saving to: ‘genres.tar.gz’

genres.tar.gz       100%[===================>]   1.14G  1.74MB/s    in 9m 39s  

2021-02-22 20:53:54 (2.02 MB/s) - ‘genres.tar.gz’ saved [1225571541/1225571541]



In [2]:
!tar --gunzip --extract --verbose --file=genres.tar.gz

genres/
genres/bextract_single.mf
genres/bl.mf
genres/blues/
genres/cl.mf
genres/classical/
genres/co.mf
genres/country/
genres/di.mf
genres/disco/
genres/hi.mf
genres/hiphop/
genres/input.mf
genres/ja.mf
genres/jazz/
genres/me.mf
genres/metal/
genres/po.mf
genres/pop/
genres/re.mf
genres/reggae/
genres/ro.mf
genres/rock/
genres/rock/rock.00000.wav
genres/rock/rock.00001.wav
genres/rock/rock.00002.wav
genres/rock/rock.00003.wav
genres/rock/rock.00004.wav
genres/rock/rock.00005.wav
genres/rock/rock.00006.wav
genres/rock/rock.00007.wav
genres/rock/rock.00008.wav
genres/rock/rock.00009.wav
genres/rock/rock.00010.wav
genres/rock/rock.00011.wav
genres/rock/rock.00012.wav
genres/rock/rock.00013.wav
genres/rock/rock.00014.wav
genres/rock/rock.00015.wav
genres/rock/rock.00016.wav
genres/rock/rock.00017.wav
genres/rock/rock.00018.wav
genres/rock/rock.00019.wav
genres/rock/rock.00020.wav
genres/rock/rock.00021.wav
genres/rock/rock.00022.wav
genres/rock/rock.00023.wav
genres/rock/rock.00024.wav
g

## Importing Libraries

In [4]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [5]:
# removing .mf files

for file in os.listdir('/content/genres'):
  if os.path.splitext(file)[1] == ".mf":
    os.remove('/content/genres/'+file)

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [7]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [11]:
file = open('music_genre_data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('music_genre_data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

The data has been extracted into a [music_genre_data.csv](https://github.com/ShaileshKumar97/Machine-Learning-Projects/blob/main/Music_Genre_Classification/music_genre_data.csv) file.

# Analysing the Data in Pandas

In [12]:
data = pd.read_csv('music_genre_data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00098.wav,0.442972,0.125422,2310.280604,2206.710936,4829.320798,0.128426,-90.677490,100.093132,-23.916267,52.581123,-20.916391,31.494152,-11.203328,20.085384,-10.909221,17.556229,-14.092525,12.988067,-11.415391,2.906069,-12.012915,4.822586,-11.156673,1.159211,-3.913956,3.250427,blues
1,blues.00033.wav,0.260669,0.133165,1430.582003,2075.602343,3246.730367,0.041970,-239.010910,108.669212,34.149349,30.242792,-0.470996,27.296513,-21.389084,14.155845,-24.505630,-1.876820,-28.052265,-11.258264,-16.484100,-7.636357,-12.796290,-15.352031,-13.732283,-7.400161,-10.083954,-13.285140,blues
2,blues.00019.wav,0.257259,0.097672,1195.411640,1481.284259,2234.878153,0.058871,-236.665344,138.409058,-4.903461,47.624584,5.179255,8.879488,0.507106,-3.818274,-14.977521,10.420425,-0.351182,0.777829,1.777306,4.235422,-0.463351,-6.105692,-5.382381,-1.337586,-3.719717,-4.353154,blues
3,blues.00059.wav,0.405961,0.169987,2235.906951,2100.004335,4390.031787,0.142288,-86.225304,91.948448,-23.408659,68.253288,-3.895873,22.745262,-9.899613,16.895977,-11.397531,18.431677,-7.015263,10.771054,-11.938685,1.005814,-5.923496,5.318165,-6.140526,1.948991,-5.102755,-0.666248,blues
4,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158825,42.351032,-6.376458,18.618876,-13.697912,15.344631,-12.285267,10.980492,-8.324325,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues


In [13]:
data.shape

(1000, 28)

In [14]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [15]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [16]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
len(y_train)

800

In [19]:
len(y_test)

200

In [20]:
X_train[10]

array([ 1.21446065, -0.12670927,  0.1801604 ,  0.80141674,  0.80410948,
       -0.74134446, -0.00482185, -0.49457792,  0.91170811,  0.4386887 ,
        0.6079677 ,  1.81629509, -0.95938884,  1.5953118 , -0.40327833,
        2.1390544 , -0.39613207,  2.13328582,  0.06497893,  1.61738906,
       -0.87527984,  1.67610547, -0.86179897,  0.61185357, -0.53751055,
       -0.00559532])

# Classification with Keras

## Building our Network

In [21]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [22]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                   

Epoch 1/20
7/7 [==============================] - 1s 4ms/step - loss: 2.2655 - accuracy: 0.1932
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 1.9248 - accuracy: 0.3598
Epoch 3/20
7/7 [==============================] - 0s 3ms/step - loss: 1.6551 - accuracy: 0.4235
Epoch 4/20
7/7 [==============================] - 0s 4ms/step - loss: 1.4864 - accuracy: 0.4934
Epoch 5/20
7/7 [==============================] - 0s 4ms/step - loss: 1.3139 - accuracy: 0.5344
Epoch 6/20
7/7 [==============================] - 0s 4ms/step - loss: 1.1458 - accuracy: 0.6136
Epoch 7/20
7/7 [==============================] - 0s 4ms/step - loss: 1.0610 - accuracy: 0.6669
Epoch 8/20
7/7 [==============================] - 0s 4ms/step - loss: 0.9972 - accuracy: 0.6832
Epoch 9/20
7/7 [==============================] - 0s 5ms/step - loss: 0.8986 - accuracy: 0.7173
Epoch 10/20
7/7 [==============================] - 0s 4ms/step - loss: 0.8676 - accuracy: 0.7331
Epoch 11/20
7/7 [======================

In [24]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 2ms/step - loss: 1.1552 - accuracy: 0.6150


In [27]:
print('test_acc: ',test_acc)

test_acc:  0.6150000095367432


Test accuracy is less than training data accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [28]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 30 epochs:

In [51]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=50,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/50
2/2 [==============================] - 1s 206ms/step - loss: 2.2827 - accuracy: 0.1492 - val_loss: 2.1030 - val_accuracy: 0.4050
Epoch 2/50
2/2 [==============================] - 0s 54ms/step - loss: 2.0714 - accuracy: 0.4078 - val_loss: 1.9395 - val_accuracy: 0.3800
Epoch 3/50
2/2 [==============================] - 0s 50ms/step - loss: 1.8777 - accuracy: 0.3999 - val_loss: 1.7871 - val_accuracy: 0.3900
Epoch 4/50
2/2 [==============================] - 0s 45ms/step - loss: 1.7012 - accuracy: 0.4182 - val_loss: 1.6716 - val_accuracy: 0.4050
Epoch 5/50
2/2 [==============================] - 0s 50ms/step - loss: 1.5644 - accuracy: 0.4633 - val_loss: 1.5928 - val_accuracy: 0.4200
Epoch 6/50
2/2 [==============================] - 0s 51ms/step - loss: 1.4459 - accuracy: 0.4871 - val_loss: 1.5392 - val_accuracy: 0.4500
Epoch 7/50
2/2 [==============================] - 0s 49ms/step - loss: 1.3575 - accuracy: 0.5128 - val_loss: 1.4785 - val_accuracy: 0.4650
Epoch 8/50
2/2 [==========

In [52]:
results

[1.4200290441513062, 0.5799999833106995]

## Predictions on Test Data

In [53]:
predictions = model.predict(X_test)

In [54]:
predictions[0].shape

(10,)

In [55]:
np.sum(predictions[0])

0.99999994

In [56]:
np.argmax(predictions[0])

5

In [58]:
preds = []
for pred in predictions:
  preds.append(np.argmax(pred))

preds = np.array(preds)

In [59]:
from sklearn.metrics import accuracy_score

print(accuracy_score(preds, y_test))

0.58
